In [13]:
from SafeTransformer import SafeTransformer
from sklearn.datasets import load_digits
import numpy as np
from sklearn.model_selection import train_test_split
data = load_digits()

In [14]:
import pandas as pd
X = pd.DataFrame(data.data)
y = pd.Series(data.target)

In [15]:
#X = X.loc[((y==7) | (y==2) | (y==4)), :]
#y = y.loc[((y==7) | (y==2) | (y==4))]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
surrogate_model = GradientBoostingClassifier().fit(X_train, y_train)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
base_model = LogisticRegression().fit(X_train, y_train)
base_predictions = base_model.predict(X_test)

In [19]:
accs = []
pens = [1]
best_score = 0
best_pen = 0
for pen in pens:
    
    safe_transformer = SafeTransformer()

    safe_transformer = safe_transformer.fit(X_train, surrogate_model, penalty = pen, pelt_model='l1')

    X_train_transformed = safe_transformer.transform(X_train)

    X_test_transformed = safe_transformer.transform(X_test)

    logistic_model_transformed = LogisticRegression()
    
    logistic_model_transformed = logistic_model_transformed.fit(X_train_transformed, y_train)

    surrogate_predictions = logistic_model_transformed.predict(X_test_transformed)
    acc = accuracy_score(y_test, surrogate_predictions)
    accs.append(acc)
    #print(mean_squared_error(y_test, standard_predictions), error)
    if acc > best_score:
        best_score = acc
        best_pen = pen

In [20]:
print(best_score)
print(accuracy_score(y_test, base_predictions))
print(accuracy_score(y_test, surrogate_model.predict(X_test)))

0.9155555555555556
0.9666666666666667
0.9755555555555555


In [21]:
print(accs)

[0.9155555555555556]


In [22]:
X_test_transformed.shape

(450, 112)

In [23]:
X_test.shape

(450, 64)

In [24]:
X_test_transformed.apply(pd.value_counts)

,"0_(0.00, +Inf)","1_(8.00, +Inf)","2_(16.00, +Inf)","3_(16.00, +Inf)","4_(16.00, +Inf)","5_(6.47, 12.48)","5_(12.48, 16.00)","5_(16.00, +Inf)","6_(16.00, +Inf)","7_(15.00, +Inf)",...,"59_(16.00, +Inf)","60_(0.46, 2.47)","60_(2.47, 7.51)","60_(7.51, 9.51)","60_(9.51, 16.00)","60_(16.00, +Inf)","61_(16.00, +Inf)","62_(16.00, +Inf)","63_(9.51, 16.00)","63_(16.00, +Inf)"
0.0,NaN,450.0,441,345,353,359,400,414,444,450.0,...,323,432,415,421,242,312,399,433,445,450.0
1.0,450.0,NaN,9,105,97,91,50,36,6,NaN,...,127,18,35,29,208,138,51,17,5,NaN
